<a href="https://colab.research.google.com/github/samiralfayed/Simple-Multilingual-Retrieval-Augmented-Generation-RAG-System/blob/main/Simple_Multilingual_Retrieval_Augmented_Generation_(RAG)_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
cd/content/drive/MyDrive/Assessment-10_Minute_School_AI_Engineer

/content/drive/MyDrive/Assessment-10_Minute_School_AI_Engineer


In [9]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.6 MB/s eta 0:00:00


In [11]:
%pip install langchain-community langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [18]:
%pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.0 MB/s et

In [1]:
%pip install langchain-google-genai

In [4]:
from google.colab import files
uploaded = files.upload()

Saving HSC26-Bangla1st-Paper.pdf to HSC26-Bangla1st-Paper.pdf


In [5]:
def load_pdf_text(pdf_path):
    loader = PyMuPDFLoader(pdf_path)
    return loader.load()

In [7]:
from getpass import getpass
import os

GOOGLE_API_KEY = getpass("Enter your Gemini API Key (will be hidden): ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter your Gemini API Key (will be hidden): ··········


In [12]:
import os
import fitz
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema import Document
import google.generativeai as genai
from google.colab import userdata


In [13]:
def extract_and_chunk_pdf(pdf_path):
    """Loads PDF, extracts text, and splits into chunks."""
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = splitter.split_documents(documents)
    return chunks

In [25]:
import shutil
def create_vectorstore(chunks, persist_directory="./chroma_db_v3"):
    """Creates and populates a Chroma vector store."""
    # Clean up old directory if needed
    if os.path.exists(persist_directory):
        shutil.rmtree(persist_directory)


    os.makedirs(persist_directory, exist_ok=True)

    embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

    vectordb = Chroma.from_documents(
        chunks,
        embedding=embedding_model,
        collection_name="bangla_pdf_collection_v3",
        persist_directory=persist_directory
    )
    return vectordb

In [21]:
def get_relevant_chunks(query, vectordb, k=5):
    """Retrieves relevant chunks from the vector store."""
    retriever = vectordb.as_retriever(search_kwargs={"k": k})
    relevant_docs = retriever.invoke(query)
    return relevant_docs

In [22]:
def answer_question_with_gemini(query, relevant_chunks):
    """Uses Gemini to answer a question based on retrieved chunks."""
    model = genai.GenerativeModel('gemini-1.5-flash-latest')

    context = "\n\n".join([doc.page_content for doc in relevant_chunks])
    prompt = f"""Based on the following text, answer the question.

Context:
{context}

Question: {query}

Answer:
"""

    response = model.generate_content(prompt)
    return response.text

In [23]:
# Example Usage
pdf_file_path = 'HSC26-Bangla1st-Paper.pdf'
user_question = "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?"

# 1. Extract and chunk the PDF
print("Extracting and chunking PDF...")
chunks = extract_and_chunk_pdf(pdf_file_path)
print(f"Created {len(chunks)} chunks.")

# 2. Create the vector store
print("Creating vector store...")
vectordb = create_vectorstore(chunks)
print("Vector store created.")

# 3. Retrieve relevant chunks
print("Retrieving relevant chunks...")
relevant_chunks = get_relevant_chunks(user_question, vectordb)
print(f"Retrieved {len(relevant_chunks)} chunks.")

# 4. Answer the question using Gemini
print("Answering question with Gemini...")
answer = answer_question_with_gemini(user_question, relevant_chunks)

print(f"\nUser Question: {user_question}")
print(f"Answer: {answer}")

Extracting and chunking PDF...
Created 115 chunks.
Creating vector store...
Vector store created.
Retrieving relevant chunks...
Retrieved 5 chunks.
Answering question with Gemini...

User Question: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
Answer: Based on the provided text,  মামা (maternal uncle) is mentioned as Anupam's  bhaagyodebota (fate/fortune god).  Question 2 states that the main element in Mama's giving Anupam good fortune is his "proभाव" (influence/impact).



Fastapi

In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
import re
import os

from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize FastAPI
app = FastAPI()


# Setup Chroma vector store
PERSIST_DIR = "./chroma_db_v3"
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectordb = Chroma(
    persist_directory=PERSIST_DIR,
    embedding_function=embedding_model,
    collection_name="bangla_pdf_collection_v3"
)

# Setup RetrievalQA chain
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-latest",
    temperature=0,
) # Added the closing parenthesis here

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

# Request model
class Question(BaseModel):
    query: str

# API route
@app.post("/ask")
async def ask_question(data: Question):
    answer = qa_chain.run(data.query)
    bangla_words = re.findall(r'[\u0980-\u09FF]+', answer)
    return {"answer": bangla_words[0] if bangla_words else ""}